In [1]:
import sys
sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld
import scipy.sparse as sps

URM_all, ICM1, ICM2, ICM3, ICM4 = ld.getCOOs()
ICML = ld.getICMlength('5km')
ICM_weighted = ld.getICMselected('7')
ICM_gen = sps.hstack([ICM1, ICM2])
ICM_all = sps.hstack([ICM1, ICM2, ICM3, ICML])
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [4]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

seed = 6180
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.7, seed=seed)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13649 ( 0.0%) Users that have less than 1 test interactions


In [13]:
import os

ofp = "../models_temp/Similarity_Hybrid/seed" + str(seed) + "/"

models_to_combine_best = {
                            'EASE': {"topK": None, "normalize_matrix": False, "l2_norm": 3749.2876352484},
                            'IALS_Hyb' : {"num_factors": 28, "epochs": 10, "confidence_scaling": "linear", "alpha": 0.43657990940994623, "epsilon": 0.35472063248578317, "reg": 0.0001698292271931609, "mw": 0.06122362507952762},
                            'RP3ICMnew': {'alpha': 1.029719677583138, 'beta': 1.0630164752134375, 'topK': 6964, 'normalize_similarity': True},
                            'RP3ICM' : {"topK": 2550, "alpha": 1.3058102610510849, "beta": 0.5150718337969987, "normalize_similarity": True, "implicit": True},
                            'IALS' :  {'num_factors': 34, 'epochs': 599, 'confidence_scaling': 'linear', 'alpha': 0.003519435539271083, 'epsilon': 0.09222402080721787, 'reg': 2.4127708108457617e-05},
                            'SLIMgensub': {"l1_ratio" : 0.025887359156206147, "topK": 2140, "alpha": 0.009567288586539689, "workers": 8, "mw": 1},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMweig': {'l1_ratio': 0.0005247075138160404, 'topK': 4983, 'alpha': 0.06067400905430761, 'workers': 8, 'mw': 2.308619939318322},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'icm_all' : {"shrink": 5675, "topK": 2310, "feature_weighting": "BM25", "normalize": False},
                            'icm_weighted': {"shrink": 4000, "topK": 985, "feature_weighting": "TF-IDF", "normalize": True},
                            'SLIM_all' : {'l1_ratio': 0.0001043005302985496, 'topK': 1135, 'alpha': 0.06322495726710943, 'workers': 8, 'mw': 1.2937257828415842},
                            'SLIM_all2' : {'topK': 4997, 'l1_ratio': 0.00036065147205858187, 'alpha': 0.03203200564199538, 'mw': 1.0785132020354402, 'workers': 8},
                            'UserKNN' : {"topK": 469, "similarity": "cosine", "shrink": 588, "normalize": True, "feature_weighting": "TF-IDF", "URM_bias": False},
                            'SLIM_BPR_Hyb' : {"epochs": 1443, "lambda_i": 8.900837513818856e-05, "lambda_j": 1.2615223007492727e-05, "learning_rate": 0.0037706733838839264, "topK": 6181, "random_seed": 1234, "sgd_mode": "sgd"},
                            'Neural': {"epochs": 295, "learning_rate": 9.598708662230564e-05, "l2_reg": 1.4122381520119457e-05, "dropout": 0.5759445339123964, "total_anneal_steps": 147387, "anneal_cap": 0.4376000433422989, "batch_size": 256, "encoding_size": 185, "next_layer_size_multiplier": 6, "max_n_hidden_layers": 1, "max_layer_size": 5000.0},
                            'EASE_stk': {"topK": None, "normalize_matrix": False, 'mw': 4.0, "l2_norm": 3749.2876352484},
                         }


In [14]:
def model_init(recommender, name, args, seed):
    path_name = ofp + name + str(seed) + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name + str(seed))
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name + str(seed))

In [15]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython_Hybrid
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIM_S_ElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommenderICM
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender_Hybrid
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender_Hybrid

In [16]:
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommenderNormal import ItemKNNSimilarityHybridRecommenderNormal
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridTwoRecommender
from Recommenders.HybridScores.DifferentStructure import TwoDifferentModelRecommender

In [17]:
groups = ld.load_groups(5)

In [18]:
recommender_slim_all = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_train, ICM_all)
recommender_slim_all2 = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_train, ICM_all)
recommender_slim_wei = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_train, ICM_weighted)
recommender_ease = EASE_R_Recommender(URM_train)
recommender_itemknn = ItemKNNCBFRecommender(URM_train, ICM_all)
recommender_rp3 = RP3betaRecommender(URM_train)
#recommender_svd = PureSVDRecommender(URM_train)
recommender_ease_hyb = EASE_R_Recommender_Hybrid(URM_train, ICM_all)

model_init(recommender_slim_all, 'SLIMall', models_to_combine_best['SLIM_all'], seed)
model_init(recommender_slim_all2, 'SLIMall2', models_to_combine_best['SLIM_all2'], seed)
model_init(recommender_slim_wei, 'SLIMweig', models_to_combine_best['SLIMweig'], seed)
model_init(recommender_ease, 'EASE', models_to_combine_best['EASE'], seed)
model_init(recommender_rp3, 'RP3beta', models_to_combine_best['RP3beta'], seed)
model_init(recommender_itemknn, 'ICM_all', models_to_combine_best['icm_all'], seed)
#model_init(recommender_svd, 'PureSVD6180', {'num_factors': 30})
model_init(recommender_ease_hyb, 'EASE_stacked', models_to_combine_best['EASE_stk'], seed)

recommender_slim_all.RECOMMENDER_NAME = 'SLIMall'
recommender_slim_all2.RECOMMENDER_NAME = 'SLIMall2'
recommender_slim_wei.RECOMMENDER_NAME = 'SLIMweig'
recommender_ease.RECOMMENDER_NAME = 'EASE'
recommender_itemknn.RECOMMENDER_NAME = 'KNN'
recommender_rp3.RECOMMENDER_NAME = 'RP3'
#recommender_svd.RECOMMENDER_NAME = 'SVD'
recommender_ease_hyb.RECOMMENDER_NAME = 'EASE_Hybrid'

../models_temp/Similarity_Hybrid/seed6180/SLIMall6180.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/seed6180/SLIMall6180'
SLIMElasticNetRecommender: Loading complete
../models_temp/Similarity_Hybrid/seed6180/SLIMall26180.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/seed6180/SLIMall26180'
SLIMElasticNetRecommender: Loading complete
../models_temp/Similarity_Hybrid/seed6180/SLIMweig6180.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/seed6180/SLIMweig6180'
SLIMElasticNetRecommender: Loading complete
../models_temp/Similarity_Hybrid/seed6180/EASE6180.zip
Model found!
EASE_R_Recommender: Loading model from file '../models_temp/Similarity_Hybrid/seed6180/EASE6180'
EASE_R_Recommender: Loading complete
../models_temp/Similarity_Hybrid/seed6180/RP3beta6180.zip
Model found!
RP3betaRecommender: Loading model from file '../models_te

In [20]:
recommender_slim_bpr = SLIM_BPR_Cython(URM_train)
recommender_slim_bpr_hyb = SLIM_BPR_Cython_Hybrid(URM_train, ICM_weighted)
recommender_ials_hyb = IALSRecommender_Hybrid(URM_train, ICM_weighted)
#recommender_p3alpha = P3alphaRecommender(URM_train)
recommender_slimer = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)

model_init(recommender_slim_bpr, 'SLIMBPR', models_to_combine_best['SLIMBPR'], seed)
model_init(recommender_slim_bpr_hyb, 'SLIMBPR_Hyb', models_to_combine_best['SLIM_BPR_Hyb'], seed)
model_init(recommender_ials_hyb, 'IALS_Hyb', models_to_combine_best['IALS_Hyb'], seed)
#model_init(recommender_p3alpha, 'P3alpha6180', models_to_combine_best['P3alpha'])
model_init(recommender_slimer, 'SLIMER', models_to_combine_best['SLIMER'], seed)

recommender_slim_bpr.RECOMMENDER_NAME = 'SLIMall'
recommender_slim_bpr_hyb.RECOMMENDER_NAME = 'SLIMall2'
recommender_ials_hyb.RECOMMENDER_NAME = 'IALS_Hyb'
#recommender_p3alpha.RECOMMENDER_NAME = 'P3alpha'
recommender_slimer.RECOMMENDER_NAME = 'SLIMER'

../models_temp/Similarity_Hybrid/seed6180/SLIMBPR6180.zip
Model found!
SLIM_BPR_Recommender: Loading model from file '../models_temp/Similarity_Hybrid/seed6180/SLIMBPR6180'
SLIM_BPR_Recommender: Loading complete
../models_temp/Similarity_Hybrid/seed6180/SLIMBPR_Hyb6180.zip
Model found!
SLIM_BPR_Recommender_Hybrid: Loading model from file '../models_temp/Similarity_Hybrid/seed6180/SLIMBPR_Hyb6180'
SLIM_BPR_Recommender_Hybrid: Loading complete
../models_temp/Similarity_Hybrid/seed6180/IALS_Hyb6180.zip
Model found!
IALSRecommender_Hybrid: Loading model from file '../models_temp/Similarity_Hybrid/seed6180/IALS_Hyb6180'
IALSRecommender_Hybrid: Loading complete
../models_temp/Similarity_Hybrid/seed6180/SLIMER6180.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/seed6180/SLIMER6180'
SLIMElasticNetRecommender: Loading complete


In [21]:
def rank_models(evaluator, recommenders):
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        print(r.RECOMMENDER_NAME, r_d.loc[10]['MAP'])
        
def rank_models_group_wise(evaluators, recommenders):
    i = 0 # group
    dict_res = {}
    for e in evaluators:
        dict_res[i] = {}
        for r in recommenders:
            r_d, _ = e.evaluateRecommender(r)
            dict_res[i][r.RECOMMENDER_NAME] = r_d.loc[10]['MAP']
        i+=1
    return dict_res

In [ ]:
def rank_models(evaluator, recommenders):
    dict_res = {}
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        dict_res[r.RECOMMENDER_NAME] = r_d.loc[10]['MAP']
        
    return dict_res

In [ ]:
import numpy as np

evaluators = []
n_groups = 5
cutoff = 10
n_users = URM_all.shape[0]

for group_id in range(0, n_groups):
    users_in_group = groups[group_id]
    print("GROUP:", group_id, "NUMBER:", len(users_in_group))
    all_users = np.arange(0, n_users)
    users_not_in_group = np.delete(all_users, users_in_group)

    evaluator_test = EvaluatorHoldout(URM_validation, cutoff_list=[cutoff], ignore_users=users_not_in_group)
    evaluators.append(evaluator_test)

In [30]:
all_rec = [recommender_slim_all, recommender_slim_all2, recommender_slim_wei, recommender_ease, recommender_itemknn, recommender_rp3, recommender_svd, recommender_slim_bpr, recommender_slim_bpr_hyb, recommender_ials_hyb, recommender_slimer, recommender_slim_all2_ease]
dict_res = rank_models(evaluator_validation, all_rec)
dict_res2 = rank_models_group_wise(evaluators, all_rec)

EvaluatorHoldout: Processed 13649 (100.0%) in 22.82 sec. Users per second: 598
             MAP
cutoff          
10      0.342705
EvaluatorHoldout: Processed 13649 (100.0%) in 20.80 sec. Users per second: 656
             MAP
cutoff          
10      0.342397
EvaluatorHoldout: Processed 13649 (100.0%) in 22.82 sec. Users per second: 598
             MAP
cutoff          
10      0.342891
EvaluatorHoldout: Processed 13649 (100.0%) in 41.39 sec. Users per second: 330
             MAP
cutoff          
10      0.338931
EvaluatorHoldout: Processed 13649 (100.0%) in 36.20 sec. Users per second: 377
             MAP
cutoff          
10      0.129848
EvaluatorHoldout: Processed 13649 (100.0%) in 28.12 sec. Users per second: 485
             MAP
cutoff          
10      0.307918
EvaluatorHoldout: Processed 13649 (100.0%) in 5.78 sec. Users per second: 2363
            MAP
cutoff         
10      0.32064
EvaluatorHoldout: Processed 13649 (100.0%) in 46.89 sec. Users per second: 291
             M

EvaluatorHoldout: Processed 4508 (100.0%) in 12.57 sec. Users per second: 359
             MAP
cutoff          
10      0.137651
EvaluatorHoldout: Processed 4508 (100.0%) in 9.73 sec. Users per second: 464
             MAP
cutoff          
10      0.332699
EvaluatorHoldout: Processed 4508 (100.0%) in 1.43 sec. Users per second: 3152
             MAP
cutoff          
10      0.346008
EvaluatorHoldout: Processed 4508 (100.0%) in 15.13 sec. Users per second: 298
             MAP
cutoff          
10      0.339687
EvaluatorHoldout: Processed 4508 (100.0%) in 22.43 sec. Users per second: 201
             MAP
cutoff          
10      0.347491
EvaluatorHoldout: Processed 4508 (100.0%) in 2.59 sec. Users per second: 1742
             MAP
cutoff          
10      0.346384
EvaluatorHoldout: Processed 4508 (100.0%) in 8.59 sec. Users per second: 525
            MAP
cutoff         
10      0.36633
EvaluatorHoldout: Processed 4508 (100.0%) in 17.12 sec. Users per second: 263
             MAP
cutoff 

In [31]:
for i in dict_res2.keys():
    dict_res2[i] = {k: v for k, v in sorted(dict_res2[i].items(), key=lambda item: item[1])}
    print(i, dict_res2[i])
    
dict_res = {k: v for k, v in sorted(dict_res.items(), key=lambda item: item[1])}
print(dict_res)

0 {'KNN': 0.154956292258151, 'SLIMall': 0.36631439029188534, 'RP3': 0.3676907376344753, 'SVD': 0.36778595760983135, 'IALS_Hyb': 0.37048981921535756, 'SLIMall2': 0.3707417632197469, 'EASE': 0.3891332096004312, 'SLIMweig': 0.39029160065852875, 'SLIMER': 0.39110664585468835, 'DifferentLossScoresHybridRecommender': 0.39143966338437985}
1 {'KNN': 0.18479301832129605, 'SLIMall2': 0.3704631329824134, 'SLIMall': 0.3744785163422692, 'RP3': 0.39152650263190086, 'SVD': 0.39240482311176406, 'IALS_Hyb': 0.39274390990329294, 'DifferentLossScoresHybridRecommender': 0.39859162688211475, 'SLIMER': 0.4009925735504139, 'SLIMweig': 0.4020430897294653, 'EASE': 0.4035332762067981}
2 {'KNN': 0.08167367853290203, 'RP3': 0.2028048620742791, 'SVD': 0.2281147807828632, 'IALS_Hyb': 0.22990222877176678, 'SLIMall': 0.23919592772384018, 'SLIMall2': 0.24800922715364426, 'EASE': 0.24834709701032495, 'SLIMweig': 0.25492782978887385, 'SLIMER': 0.2551878178455849, 'DifferentLossScoresHybridRecommender': 0.256367766605024

In [24]:
import numpy as np

def test_percentage_sim(recommender_a, recommender_b, evaluator, high1, low1, step):
    recommender = ItemKNNSimilarityHybridRecommenderNormal(URM_train, recommender_a.W_sparse, recommender_b.W_sparse)
    results = []
    alp_space = np.linspace(high1, low1, step, False)
    for alp in alp_space:
        recommender.fit(alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

def test_percentage_sco(recommender_a, recommender_b, evaluator, high1, low1, step, norm=None):
    recommender = TwoDifferentModelRecommender(URM_train, recommender_a, recommender_b)
    results = []
    alp_space = np.linspace(high1, low1, step, False)
    for alp in alp_space:
        recommender.fit(norm, alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

In [25]:
alp_space, results = test_percentage_sco(recommender_slimer, recommender_ease_hyb, evaluator_validation, 0.3, 0.7, 50) # 0.588 : 0.344262

Deallocating Cython objects
EvaluatorHoldout: Processed 13649 (100.0%) in 40.36 sec. Users per second: 338
             MAP
cutoff          
10      0.343505
0.3 : 0.34350482098229473
EvaluatorHoldout: Processed 13649 (100.0%) in 37.43 sec. Users per second: 365
            MAP
cutoff         
10      0.34359
0.308 : 0.34359016649862456
EvaluatorHoldout: Processed 13649 (100.0%) in 38.61 sec. Users per second: 354
             MAP
cutoff          
10      0.343601
0.316 : 0.34360149647570287
EvaluatorHoldout: Processed 13649 (100.0%) in 37.93 sec. Users per second: 360
             MAP
cutoff          
10      0.343676
0.324 : 0.3436764191108865
EvaluatorHoldout: Processed 13649 (100.0%) in 41.13 sec. Users per second: 332
             MAP
cutoff          
10      0.343747
0.33199999999999996 : 0.34374732086890825
EvaluatorHoldout: Processed 13649 (100.0%) in 56.37 sec. Users per second: 242
             MAP
cutoff          
10      0.343758
0.33999999999999997 : 0.3437579152842199
Eva

EvaluatorHoldout: Processed 13649 (100.0%) in 37.07 sec. Users per second: 368
             MAP
cutoff          
10      0.344097
0.6839999999999999 : 0.34409688133440985
EvaluatorHoldout: Processed 13649 (100.0%) in 37.06 sec. Users per second: 368
             MAP
cutoff          
10      0.344138
0.692 : 0.34413775589118356


In [26]:
alp_space, results = test_percentage_sco(recommender_slim_all2, recommender_ease_hyb, evaluator_validation, 0.3, 0.7, 50) # 0.532 : 0.34449

EvaluatorHoldout: Processed 13649 (100.0%) in 36.04 sec. Users per second: 379
             MAP
cutoff          
10      0.343987
0.3 : 0.3439873931109577
EvaluatorHoldout: Processed 13649 (100.0%) in 35.09 sec. Users per second: 389
             MAP
cutoff          
10      0.344002
0.308 : 0.3440017467411446
EvaluatorHoldout: Processed 13649 (100.0%) in 34.93 sec. Users per second: 391
             MAP
cutoff          
10      0.344125
0.316 : 0.34412547230043244
EvaluatorHoldout: Processed 13649 (100.0%) in 34.84 sec. Users per second: 392
            MAP
cutoff         
10      0.34419
0.324 : 0.3441897045774638
EvaluatorHoldout: Processed 13649 (100.0%) in 34.76 sec. Users per second: 393
             MAP
cutoff          
10      0.344132
0.33199999999999996 : 0.34413235111125223
EvaluatorHoldout: Processed 13649 (100.0%) in 35.04 sec. Users per second: 389
             MAP
cutoff          
10      0.344156
0.33999999999999997 : 0.34415554601942067
EvaluatorHoldout: Processed 1364